In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
test = pd.read_csv('C:/Users/adith/Desktop/Datasets/emnist-letters-test.csv')
train = pd.read_csv('C:/Users/adith/Desktop/Datasets/emnist-letters-train.csv')
train.head()

,23,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.406,0.407,0.408,0.409,0.410,0.411,0.412,0.413,0.414,0.415
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(test.shape)
print(train.shape)

(14799, 785)
(88799, 785)


In [4]:
y1 = np.array(train.iloc[:,0].values)
x1 = np.array(train.iloc[:,1:].values)
y2 = np.array(test.iloc[:,0].values)
x2 = np.array(test.iloc[:,1:].values)
print(y1.shape)
print(x1.shape)
print(y2.shape)
print(x2.shape)

(88799,)
(88799, 784)
(14799,)
(14799, 784)


In [5]:
x = x1[np.any([y1==1,y1==2],axis=0)]
y = y1[np.any([y1==1,y1==2],axis=0)]
x_t = x2[np.any([y2==1,y2==2],axis=0)]
y_t = y2[np.any([y2==1,y2==2],axis=0)]

In [6]:
#Data Normalisation
X_train_normalised = x/255.0
X_test_normalised = x_t/255.0

In [7]:
X_train_tr = X_train_normalised.transpose()
Y_train_tr = y.reshape(1,y.shape[0])
X_test_tr = X_test_normalised.transpose()
Y_test_tr = y_t.reshape(1,y_t.shape[0])
print(X_train_tr.shape)
print(Y_train_tr.shape)
print(X_test_tr.shape)
print(Y_test_tr.shape)

train_dimension = X_train_tr.shape[1]
test_dimension = X_test_tr.shape[1]

(784, 6792)
(1, 6792)
(784, 1599)
(1, 1599)


In [8]:
X_train = X_train_tr
Y_train = Y_train_tr-1
X_test = X_test_tr
Y_test = Y_test_tr-1

In [9]:
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-z))
    
    return s

In [10]:
def initialize(dim):
    w = np.zeros((dim,1))
    b=0
    
    assert (w.shape == (dim,1))
    assert (isinstance(b, float) or isinstance(b,int))
    
    return w,b

In [11]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]
    
    z = np.dot(w.T,X)+b
    A = sigmoid(z)
    cost = -1.0/m*np.sum(Y*np.log(A)+(1.0-Y)*np.log(1.0-A))
    
    dw = 1.0/m*np.dot(X, (A-Y).T)
    db = 1.0/m*np.sum(A-Y)
    
    cost = np.squeeze(cost)
    
    grads = {"dw": dw, 
             "db":db}
    
    return grads, cost

In [12]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        #if i % 100 == 0:
        costs.append(cost)
            
        #if print_cost and i % 100 == 0:
        print ("Cost (iteration %i) = %f" %(i, cost))
            
    grads = {"dw": dw, "db": db}
    params = {"w": w, "b": b}
        
    return params, grads, costs

In [13]:
def predict (w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0],1)
    
    A = sigmoid (np.dot(w.T, X)+b)
    
    for i in range(A.shape[1]):
        if (A[:,i] > 0.5): 
            Y_prediction[:, i] = 1
        elif (A[:,i] <= 0.5):
            Y_prediction[:, i] = 0
            
    assert (Y_prediction.shape == (1,m))
    
    return Y_prediction

In [14]:
def model (X_train, Y_train, X_test, Y_test, num_iterations = 1000, learning_rate = 0.5, print_cost = False):
    w, b = initialize(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict (w, b, X_test)
    Y_prediction_train = predict (w, b, X_train)
    
    train_accuracy = 100.0 - np.mean(np.abs(Y_prediction_train-Y_train)*100.0)
    test_accuracy = 100.0 - np.mean(np.abs(Y_prediction_test-Y_test)*100.0)
    
    d = {"costs": costs,
        "Y_prediction_test": Y_prediction_test,
        "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}
    
    print ("Accuarcy Test: ",  test_accuracy)
    print ("Accuracy Train: ", train_accuracy)
    
    return d

In [15]:
d = model(X_train ,Y_train ,X_test ,Y_test ,num_iterations = 10 ,learning_rate = 0.3, print_cost = True)

Cost (iteration 0) = 0.693147
Cost (iteration 1) = 0.441301
Cost (iteration 2) = 0.398686
Cost (iteration 3) = 0.533168
Cost (iteration 4) = 0.822792
Cost (iteration 5) = 0.954179
Cost (iteration 6) = 0.600914
Cost (iteration 7) = 0.370509
Cost (iteration 8) = 0.234144
Cost (iteration 9) = 0.201446
Accuarcy Test:  93.62101313320825
Accuracy Train:  92.99175500588927
